In [4]:
# Cargar Celda
from DynamicCodeLoader import loadDynamicCode

model = loadDynamicCode("JuegoEscapaManual.ipynb", "Model")
search = loadDynamicCode("Busqueda.ipynb", "Search")

In [5]:
from Loader import Loader

loader = Loader()
files = loader.get_all_levels()
txt_level = loader.get_txt_level(files[0])

level, state = loader.load_level(txt_level)

In [6]:
txt_level

['#######', '#x* ###', '#x + -#', '#x   @#', '#!¡   #', '#######']

In [7]:
print(state)

print(f"\nIs goal: {model.is_goal(state)}")

new_state = model.move(level, state, [0, -1])
print(new_state)


-> Player: [3, 5]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

Is goal: False

-> Player: [3, 4]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False


In [8]:
from Gui import Gui
from ipywidgets import HTML
ui = Gui()

htmlstr = ui.get_html(level, state)
HTML(value=htmlstr)

HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

In [9]:
board = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
         [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
         [1, 0, 1, 0, 1, 1, 1, 1, 0, 1],
         [1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
         [1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 1],
         [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
         [1, 0, 1, 0, 0, 0, 0, 0, 0, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

initialStatePF = (1, 1)
finalStatePF = (2, 5)


initialNodePF = search.createInitialNodePF(initialStatePF, finalStatePF, board, search.heuristicPF)

sol = search.AStar(initialNodePF, search.successorsPF, search.isGoal, search.heuristicPF)

print(sol)

[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 5), (7, 6), (7, 7), (7, 8), (6, 8), (5, 8), (4, 8), (3, 8), (2, 8), (2, 7), (2, 6), (2, 5)]


In [10]:
# Cargar Celda

iterations = 0
total_cost = 0
global_level = None

def initial_node_JE(level, state, fH):
    global global_level
    global iterations

    global_level = level
    iterations = 0

    initialNode = search.Node(state, None, 0, fH(state))

    return initialNode

In [11]:
# Cargar Celda

def goal_JE(node):
    global total_cost

    state = node.getState()

    total_cost = node.getG()

    return model.is_goal(state)

In [12]:
# Cargar Celda

def heuristic_JE(state):
    global global_level
    global iterations
    iterations += 1

    player = state.get_player()
    if not player:
        return 0

    # Manhattan
    xPlayer, yPlayer = player
    xTarget, yTarget = global_level.get_target()
    cost = abs(xPlayer - xTarget) + abs(yPlayer - yTarget)

    if not state.get_player_has_key():
        cost += 1000

    return cost


In [13]:
# Cargar Celda

def successors_JE(node, hSuc):
    global global_level
    movs = ((1, 0), (-1, 0), (0, 1), (0, -1))
    childs = []

    state = node.getState()
    g = node.getG()
    
    for mov in movs:
        child = model.move(global_level, state, mov)
        if child != state:
            h = hSuc(child)
            new_g = g + 1
            new_f = new_g + h
            childs.append(search.Node(child, node, new_g, new_f))

    return childs

In [14]:
initialNode = initial_node_JE(level, state, heuristic_JE)

initialNode

Estado Nodo: 
-> Player: [3, 5]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False 
F Nodo: 1005

In [15]:
successors = successors_JE(initialNode, heuristic_JE)

successors

[Estado Nodo: 
 -> Player: [4, 5]
 -> Rocks: {(2, 3)}
 -> Water: {(2, 5)}
 -> Crosses: {(3, 1), (1, 1), (2, 1)}
 -> Keys: {(4, 2)}
 -> Player Has Key: False 
 F Nodo: 1007,
 Estado Nodo: 
 -> Player: [3, 4]
 -> Rocks: {(2, 3)}
 -> Water: {(2, 5)}
 -> Crosses: {(3, 1), (1, 1), (2, 1)}
 -> Keys: {(4, 2)}
 -> Player Has Key: False 
 F Nodo: 1005]

In [16]:
sol = search.AStar(initialNode, successors_JE, goal_JE, heuristic_JE)

print("Solución: ")
for state in sol:
    print(state)
print(f"\nNodos evaluados: {iterations}")
print(f"Coste de la solución: {total_cost}")

Solución: 

-> Player: [3, 5]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [3, 4]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [3, 3]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [3, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [4, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: set()
-> Player Has Key: True

-> Player: [3, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: set()
-> Player Has Key: True

-> Player: [2, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: set()
-> Player Has Key: True

-> Player: None
-> Rocks: {(2, 3)}
-> Wat

In [17]:
from ipywidgets import VBox, Label, Layout, Button

html = []
for st in sol:
    print(st)
    htmlstr = ui.get_html(level, st)
    html.append(HTML(value=htmlstr))

box_layout = Layout(height="400px",
                    flex_direction="column",
                    display="flex",
                    overflow_y="scroll")
carousel = VBox(children=html, layout=box_layout)
VBox([carousel])


-> Player: [3, 5]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [3, 4]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [3, 3]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [3, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: {(4, 2)}
-> Player Has Key: False

-> Player: [4, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: set()
-> Player Has Key: True

-> Player: [3, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: set()
-> Player Has Key: True

-> Player: [2, 2]
-> Rocks: {(2, 3)}
-> Water: {(2, 5)}
-> Crosses: {(3, 1), (1, 1), (2, 1)}
-> Keys: set()
-> Player Has Key: True

-> Player: None
-> Rocks: {(2, 3)}
-> Water: {(2, 5)

In [18]:
from IPython.display import display
from MediadorVPedro import Mediator
from Gui import Gui

import warnings
warnings.filterwarnings("ignore")
ui = Gui(manual = False)

med = Mediator.get_instance(modelPath = "JuegoEscapaManual.ipynb", aStarPath="Busqueda.ipynb", nodesPath="JuegoEscapaAutomatico.ipynb")   
med.register_ui(ui)


# Cuando llamo a la función se crean
display(ui.get_ui_elements())

<module 'Model'>


In [19]:
# AÑADIR ANIMACIÓN AL RESOLVER Y MEJORAR OTROS ASPECTOS Y DOCUMENTAR ATOPE! MEJORAR OPTIMIZACIÓN
